In [1]:
test_semantic_model_url = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/semantic_model.txt' 
questions_url           = 'https://raw.githubusercontent.com/djouallah/semantic_sql_testing/refs/heads/main/questions.json'
SF = 1
output_dir="/tmp/llm"

In [2]:
import requests
import json
import duckdb
import pathlib
import time
import pandas as pd
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os


# Generate Data

In [3]:
if SF <1 :
 schema = f"{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Run Tests

In [4]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        print(f"Error fetching system prompt from GitHub ({test_semantic_model_url}): {e}")
        return f"Error fetching system prompt: {e}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:11434/api/chat'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with Ollama API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from Ollama: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama', 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"


    cleaned_text = re.sub(r'```(sql|duckdb)?\s*([\s\S]*?)\s*```', r'\2', generated_text, flags=re.IGNORECASE).strip()

    if "```" in cleaned_text or "SELECT " in cleaned_text.upper() or "WITH " in cleaned_text.upper():
         cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()


    return cleaned_text

In [5]:
def execute_sql_with_retry(query, test_model,provider, api_key, azure_endpoint, azure_api_version,max_attempts):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        try:
            # Execute the SQL query
            result = con.sql(current_query).df()
            # Return the DataFrame result and the number of attempts
            return result, attempt
        except Exception as e:
            # Catch any exception during execution
            error_message = str(e).lower()

            # Check if the error is likely a syntax, parser, or binder error
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                # If it's a non-syntax error, return the error message and attempts
                return f"Non-syntax error: {e}", attempt

            # If it's a syntax-like error, print attempt info
            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {e}")

            # If max attempts reached, return the final error message and attempts
            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {e}", attempt

            # Prepare message for Gemini to fix the query
            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {e}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            # Get corrected query from Gemini
            corrected_query = get_ai_response(message, test_model,provider, api_key, azure_endpoint, azure_api_version)

            # If Gemini returns an error, return that error message and attempts
            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt

            #print(f"Received corrected query")
            # Update current_query with the corrected one and increment attempt
            current_query = corrected_query.strip()
            attempt += 1
    return "Unexpected error or loop termination", attempt



In [6]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            # Attempt to execute the SQL query
            # Ensure execute_sql_with_retry returns a DataFrame on success or a string error
            # Pass the test_model to execute_sql_with_retry as it might need it
            result_from_execution, attempts_count = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version, max_attempts=3)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, # Changed key name slightly for JSON convention
            "sql_query": sql_query_or_error, # Store the generated query string (could be AI error)
            "attempts": attempts_count,
            "result": query_result_data_json, # This is now a list of dictionaries
            "result_count": result_row_count, # Use the calculated row count
            "error_details": error_details # Store error details
        })

    os.makedirs(output_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(output_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [7]:
try:
    response = requests.get(questions_url)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
    questions = json.loads(response.text)

    print("Successfully retrieved questions:")
except requests.exceptions.RequestException as e:
    print(f"Error retrieving file from GitHub: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Successfully retrieved questions:


In [8]:
%%time
ask_question(questions,"gpt-4o","azure_foundry",os.getenv("LLM_API_KEY"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

Question 1: What is the overall total sales revenue?
-- Overall total sales revenue
SELECT 
    SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
FROM store_sales AS ss;


,total_sales
0,5.138666e+09


Execution: SUCCESS

Execution Time: 2.56 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
-- What is the total number of items sold across all transactions?
SELECT 
    SUM(ss.ss_quantity) AS total_quantity_sold
FROM store_sales AS ss;


,total_quantity_sold
0,138963631.0


Execution: SUCCESS

Execution Time: 5.09 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- What is the total monetary value of all returned items?
SELECT 
    SUM(sr.sr_return_amt) AS total_returns
FROM store_returns AS sr;


...

,item_class,customer_type,avg_net_sales_per_transaction
0,wallpaper,None,2182.882320
1,decor,None,2172.952359
2,stereo,None,2145.389483
3,fishing,None,2100.967304
4,custom,None,2078.424679
...,...,...,...
295,flatware,None,1757.351243
296,musical,None,1756.913914
297,sports,None,1747.283095
298,camping,None,1741.471667


Execution: SUCCESS

Execution Time: 6.17 seconds
 ############################### 
Question 20: show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
-- show all stores net sales, average net sales, and percentage of net sales compared to average net sales for each store
WITH store_sales_agg AS (
    SELECT
        st.s_store_name,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
    GROUP BY st.s_store_name
), store_returns_agg AS (
    SELECT
        st.s_store_name,
        SUM(sr.sr_return_amt) AS total_returns
    FROM store_returns AS sr
    INNER JOIN store AS st ON sr.s_store_sk = st.s_store_sk
    GROUP BY st.s_store_name
), store_net_sales AS (
    SELECT
        COALESCE(ss.s_store_name, sr.s_store_name) AS store_name,
        COALESCE(ss.total_sales, 0) - COALESC

'Max attempts reached. Last error: Binder Error: Referenced table "sr" not found!\nCandidate tables: "store_sales_agg"'

Execution: FAILED

Execution Time: 15.50 seconds
 ############################### 
CPU times: total: 35.2 s
Wall time: 2min 6s


'Successfully processed 20 questions. Results saved to /tmp/llm\\20250510_154008_gpt-4o.json'

In [9]:
%%time
ask_question(questions,'qwen3:8b',"ollama")

Question 1: What is the overall total sales revenue?
-- What is the overall total sales revenue?
SELECT
  total_sales
FROM
  (SELECT SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
   FROM store_sales ss) AS total_sales_subquery;


,total_sales
0,5.138666e+09


Execution: SUCCESS

Execution Time: 32.04 seconds
 ############################### 
Question 2: What is the total number of items sold across all transactions?
-- What is the total number of items sold across all transactions?
SELECT total_quantity FROM store_sales;
-- What is the total number of items sold across all transactions?
SELECT total_quantity FROM store_sales;
Attempt 1/3 failed with syntax error: Binder Error: Referenced column "total_quantity" not found in FROM clause!
Candidate bindings: "ss_quantity", "ss_ext_discount_amt", "ss_coupon_amt"


,total_quantity
0,138963631.0


Execution: SUCCESS

Execution Time: 10.14 seconds
 ############################### 
Question 3: What is the total monetary value of all returned items?
-- What is the total monetary value of all returned items?
SELECT
  total_returns
FROM
  (SELECT
    SUM(sr.sr_return_amt) AS total_returns
  FROM
    store_returns sr) AS returns;


...

,item_class,customer_type,avg_net_sales_per_transaction
0,wallpaper,None,2182.882320
1,decor,None,2172.952359
2,stereo,None,2145.389483
3,fishing,None,2100.967304
4,custom,None,2078.424679
...,...,...,...
295,flatware,None,1757.351243
296,musical,None,1756.913914
297,sports,None,1747.283095
298,camping,None,1741.471667


Execution: SUCCESS

Execution Time: 31.76 seconds
 ############################### 
Question 20: show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
-- show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending
WITH store_sales_agg AS (
    SELECT
        st.s_store_sk,
        st.s_store_name,
        SUM(ss.ss_sales_price * ss.ss_quantity) AS total_sales
    FROM store_sales AS ss
    INNER JOIN store AS st ON ss.ss_store_sk = st.s_store_sk
    GROUP BY st.s_store_sk, st.s_store_name
), store_returns_agg AS (
    SELECT
        st.s_store_sk,
        st.s_store_name,
        SUM(sr.sr_return_amt) AS total_returns
    FROM store_returns AS sr
    INNER JOIN store AS st ON sr.sr_store_sk = st.s_store_sk
    GROUP BY st.s_store_sk, st.s_store_name
), net_sales AS (
   

,s_store_name,net_sales,avg_net_sales,percentage_of_avg
0,bar,8.044211e+08,4.009164e+08,200.645571
1,eing,8.028028e+08,4.009164e+08,200.241944
2,ese,8.023467e+08,4.009164e+08,200.128165
3,able,8.014186e+08,4.009164e+08,199.896669
4,ation,8.007528e+08,4.009164e+08,199.730609
5,ought,7.992551e+08,4.009164e+08,199.357041
6,ought,0.000000e+00,4.009164e+08,0.000000
7,anti,0.000000e+00,4.009164e+08,0.000000
8,eing,0.000000e+00,4.009164e+08,0.000000
9,ought,0.000000e+00,4.009164e+08,0.000000


Execution: SUCCESS

Execution Time: 86.03 seconds
 ############################### 
CPU times: total: 13 s
Wall time: 19min 8s


'Successfully processed 20 questions. Results saved to /tmp/llm\\20250510_154008_qwen3_8b.json'

# Check Results

In [16]:
duckdb.sql(f""" from '{output_dir}/*.json' order by nbr,model """).to_view("results")
try:
 duckdb.sql(f""" load excel ;copy results to '{output_dir}/test.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")

In [37]:
# check number of rows returned by each model
duckdb.sql(f""" 
           pivot( select nbr,question,model,[result_count , cardinality(result[1])] as result_shape from results ) 
           on model using min(result_shape) order by nbr
        """).show(max_width=120)

┌───────┬──────────────────────────────────────────────────────────────────────────────────────┬───────────┬───────────┐
│  nbr  │                                       question                                       │  gpt-4o   │ qwen3:8b  │
│ int64 │                                       varchar                                        │ int128[]  │ int128[]  │
├───────┼──────────────────────────────────────────────────────────────────────────────────────┼───────────┼───────────┤
│     1 │ What is the overall total sales revenue?                                             │ [1, 1]    │ [1, 1]    │
│     2 │ What is the total number of items sold across all transactions?                      │ [1, 1]    │ [1, 1]    │
│     3 │ What is the total monetary value of all returned items?                              │ [1, 1]    │ [1, 1]    │
│     4 │ List the names of all stores, order by store name.                                   │ [12, 1]   │ [12, 1]   │
│     5 │ What are the different

Manual check for smilar results ( same rows and columns)

In [21]:
manual = duckdb.sql("""
   with   zzzzz AS(
    SELECT 
        nbr, 
        model, 
        result
    FROM results 
    WHERE nbr not in ( select nbr from results group by all having count(distinct(cardinality(result[1]))) > 1 or count(distinct(result_count )) > 1 order by nbr))
pivot(from zzzzz ) on model using min(result) order by nbr
""")
manual.show(max_width=250)
try:
 duckdb.sql(f""" load excel ;copy manual to '{output_dir}/check.xlsx' (format 'xlsx', header 'true', overwrite)  """)
except Exception as e:
    print(f"Error exporting to Excel: {e}")
    print("close the excel file or tey again.")

┌───────┬──────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  nbr  │        gpt-4o        │                                                                                                        qwen3:8b                                                                                                         │
│ int64 │ map(varchar, json)[] │                                                                                                  map(varchar, json)[]                                                                                                   │
├───────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────